In [47]:
!pip install -q requests
!pip install streamlit-lottie
!pip install ntscraper
!pip install re

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [65]:
%%writefile app.py
import re
import json
import requests
from streamlit_lottie import st_lottie
import pickle
import streamlit as st
from ntscraper import Nitter
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt

# Load tokenizer object
with open('tokenizer.pickle', 'rb') as handle:
    tokenizers = pickle.load(handle)

models = {
    'LASTM 1 LAYER MODEL': load_model('lastm-1-layer-best_model.h5'),
    'LASTM 2 LAYER MODEL': load_model('lastm-2-layer-best_model.h5'),
    'GRU MODEL': load_model('gru-best_model.h5'),
    'CNN + LSTM MODEL': load_model('cnn+lastm-best_model.h5'),
}

st.set_page_config(layout="wide")

def anime(url):
  url_anime = requests.get(url)
  if url_anime.status_code == 200:

        return url_anime.json()

def remove_special_characters(text):
    # Define the pattern for special characters
    pattern = r'[^\w\s]'

    # Remove special characters using regex
    return re.sub(pattern,'',text)

c1,c2,c3=st.columns([3,9,3])


with c1:

  st_lottie(anime("https://lottie.host/fd9e988f-4b8d-4a6e-bf50-9da5bc740a68/080cflQZMj.json"))


with c3:

  st_lottie(anime("https://lottie.host/1406ce73-529a-49e6-a0fd-42da9479f2f2/MXEeOoXTvY.json"))

with c2:
  st.title("Suicide Detection Web App")
  st.markdown("<h2 style='color: black; font-style: normal ;text-decoration: underline'>Enter Twitter username</h2>", unsafe_allow_html=True)
  text = st.text_area("")

  if st.button("Predict"):
    if text:
#Get the latest 100 tweets from a single user (@elonmusk in this example)
      tweets = Nitter().get_tweets(text, mode='user', number=2)
# Extract content from each tweet
      tweet_contents = []
      for tweet in tweets["tweets"]:
        tweet_contents.append(tweet["text"])

      predictions = {}
      max_percentage = 0
      most_likely_prediction = ''
      for twt in tweet_contents:

        i=1
        for model_name, model in models.items():
          # Convert NumPy array to string
          twt_str = ''.join([str(token) for token in twt])
          twt_str = re.sub(r'#[^\s]+|@[^\s]+|:\S+','',remove_special_characters(twt_str))
          if i==1:
            st.markdown(f"<h1 style='font-size: 24px; color: black;'>Tweets fetched: {twt_str}</h1>", unsafe_allow_html=True)
            i=0
          twt_seq = tokenizers.texts_to_sequences([twt_str])
          twt_seq = pad_sequences(twt_seq, maxlen=60, dtype='int32')

          predicted = model.predict(twt_seq, batch_size=1, verbose=True)
          percentage = predicted[0][0]  # Percentage of Potential Suicide Post

          if percentage > max_percentage:
              max_percentage = percentage
              most_likely_prediction = "Suicide" if percentage > 0.5 else "Non-Suicide"

      st.markdown("<h2 style='font-style:normal;color:black;text-decoration: underline'>Pie Chart</h3>", unsafe_allow_html=True)

      # Create and display a pie chart
      fig, ax = plt.subplots()
      labels = ['Potential Suicide', 'Non-Suicide']
      sizes = [max_percentage, 1 - max_percentage]
      ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
      ax.axis('equal')
      st.pyplot(fig)

      st.markdown("<h2 style='font-style: Normal ;color :black ;text-decoration: underline'>Prediction</h2>", unsafe_allow_html=True)

       # Display the most likely prediction in large font
      st.markdown(f"<h1 style='font-size: 24px; color: black;'>Most Likely Prediction: {most_likely_prediction}</h1>", unsafe_allow_html=True)






Overwriting app.py


In [66]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.49s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.105.88.220
npx: installed 22 in 2.616s
your url is: https://big-parents-check.loca.lt
